In [5]:
import os 

import geopandas as gpd
import fiona
import rasterio
import xarray as xr
import re
import rtree
import shapely
import pickle
#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from xml.etree import ElementTree

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset
path_to_complete_dataset = "verified/complete_dataset"
path_to_tiles_folder_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"tiles")
os.makedirs(path_to_tiles_folder_complete_dataset, exist_ok=True)

In [4]:
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
fiona.listlayers(quad_indicies_path)

['Cell_Features',
 'Cell_History',
 'CellGrid_15Minute',
 'CellGrid_1X1Degree',
 'CellGrid_1X2Degree',
 'CellGrid_3_75Minute',
 'CellGrid_30X60Minute',
 'CellGrid_7_5Minute',
 'Cell_PolygonAll']

In [3]:
dataset = rasterio.open(os.path.join(path_to_tiles_folder_complete_dataset, os.listdir(path_to_tiles_folder_complete_dataset)[0]))
left = dataset.bounds[0]
right = dataset.bounds[2]
bottom = dataset.bounds[1]
top = dataset.bounds[3]
x = dataset.shape[0] 
y = dataset.shape[1] 

np.linspace(left, right, num=x-1)# endpoint=True, retstep=False, dtype=None, axis=0)

array([453672.        , 453672.50487684, 453673.00975368, ...,
       459778.99024632, 459779.49512316, 459780.        ])

In [4]:
# Read the data
## Get tile locations
da = xr.open_rasterio(os.path.join(path_to_tiles_folder_complete_dataset, os.listdir(path_to_tiles_folder_complete_dataset)[0]))
# Compute the lon/lat coordinates with rasterio.warp.transform
ny, nx = len(da['y']), len(da['x'])
x, y = np.meshgrid(da['x'], da['y'])
da.crs

'+init=epsg:26911'

In [27]:
tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")
image_characteristics = pd.read_csv("image_characteristics.csv")

#for unique_standard_tile_names in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]:
#    if unique

In [40]:
image_characteristics

,Unnamed: 0,state,resolution,year,capture_date,utm_zone,standard_tile_name,six_digit_chip_name,NW_pixel_coordinates,SE_pixel_coordinates,row_indicies,col_indicies
0,0,ca,060cm,2018,20180822,11,m_3411761_sw_11_060_20180822,ca_060cm_2018_34117_m_3411761_sw_11_060_201808...,"[0, 3072]","[511, 3583]",6,0
1,1,ca,060cm,2018,20180822,11,m_3411761_sw_11_060_20180822,ca_060cm_2018_34117_m_3411761_sw_11_060_201808...,"[3072, 4608]","[3583, 5119]",9,6
2,2,ca,060cm,2018,20180822,11,m_3411761_sw_11_060_20180822,ca_060cm_2018_34117_m_3411761_sw_11_060_201808...,"[4608, 4096]","[5119, 4607]",8,9
3,3,ca,060cm,2018,20180822,11,m_3411761_sw_11_060_20180822,ca_060cm_2018_34117_m_3411761_sw_11_060_201808...,"[7168, 1536]","[7679, 2047]",3,14
4,4,ca,060cm,2018,20180822,11,m_3411761_sw_11_060_20180822,ca_060cm_2018_34117_m_3411761_sw_11_060_201808...,"[7168, 6656]","[7679, 7167]",13,14
...,...,...,...,...,...,...,...,...,...,...,...,...
11502,11502,ky,060cm,2018,20180904,16,m_3808543_ne_16_060_20180904,ky_060cm_2018_38085_m_3808543_ne_16_060_201809...,"[3072, 512]","[3583, 1023]",1,6
11503,11503,ky,060cm,2018,20180904,16,m_3808543_ne_16_060_20180904,ky_060cm_2018_38085_m_3808543_ne_16_060_201809...,"[5120, 4096]","[5631, 4607]",8,10
11504,11504,ky,060cm,2018,20180904,16,m_3808543_ne_16_060_20180904,ky_060cm_2018_38085_m_3808543_ne_16_060_201809...,"[7168, 512]","[7679, 1023]",1,14
11505,11505,ky,060cm,2018,20180904,16,m_3808543_ne_16_060_20180904,ky_060cm_2018_38085_m_3808543_ne_16_060_201809...,"[8704, 2048]","[9215, 2559]",4,17


In [42]:
tiles_with_no_images = 0
for standard_tile_name in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]:
    images_in_tile = image_characteristics.loc[image_characteristics["standard_tile_name"] == standard_tile_name]
    if images_in_tile.empty:
        tiles_with_no_images += 1 
    #else:
         
        #print(images_in_tile)

In [53]:
os.path.abspath(image_characteristics.loc[image_characteristics["standard_tile_name"] == tile_names_tile_urls_complete_array_unique_standard_tile_names[1000,2]]["six_digit_chip_name"][9296])


'C:\\Users\\rapiduser\\cred\\AST_dataset\\data_download_and_preprocessing\\nm_060cm_2018_35108_m_3510837_ne_12_060_20180508_000079.jpg'

In [30]:
image_characteristics["standard_tile_name"]

0        m_3411761_sw_11_060_20180822
1        m_3411761_sw_11_060_20180822
2        m_3411761_sw_11_060_20180822
3        m_3411761_sw_11_060_20180822
4        m_3411761_sw_11_060_20180822
                     ...             
11502    m_3808543_ne_16_060_20180904
11503    m_3808543_ne_16_060_20180904
11504    m_3808543_ne_16_060_20180904
11505    m_3808543_ne_16_060_20180904
11506    m_3808543_ne_16_060_20180904
Name: standard_tile_name, Length: 11507, dtype: object

In [25]:
tile_names_tile_urls_complete_array_unique_standard_tile_names[tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]=='m_3411761_sw_11_060_20180822']

array([], shape=(0, 4), dtype='<U120')

In [ ]:
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
df.loc[df['column_name'].isin(some_values)]


In [ ]:

## Combine tiles

- For every tile
- identify all annotated images (.xmls) corresponding to the tile
    - identify the position of the image
    

def newRunRun(folder):
    xml_files = glob.glob(folder+"/*.xml")
    node = None
    for xmlFile in xml_files:      
        tree = ElementTree.parse(xmlFile)
        root = tree.getroot()
        if node is None:
            node = root
        else:
            elements = root.find("./results")           
            for element in elements._children:
                node[1].append(element)                
    print ElementTree.tostring(node)

folder = "resources"
newRunRun(folder) 

In [ ]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO

# Replace <Subscription Key> with your valid subscription key.
subscription_key = "f244aa59ad4f4c05be907b4e78b7c6da"
assert subscription_key

vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v2.0/"

ocr_url = vision_base_url + "ocr"

# Set image_url to the URL of an image that you want to analyze.
image_url = "https://cdn-ayb.akinon.net/cms/2019/04/04/e494dce0-1e80-47eb-96c9-448960a71260.jpg"

headers = {'Ocp-Apim-Subscription-Key': subscription_key}
params  = {'language': 'unk', 'detectOrientation': 'true'}
data    = {'url': image_url}
response = requests.post(ocr_url, headers=headers, params=params, json=data)
response.raise_for_status()

analysis = response.json()

# Extract the word bounding boxes and text.
line_infos = [region["lines"] for region in analysis["regions"]]
word_infos = []
for line in line_infos:
    for word_metadata in line:
        for word_info in word_metadata["words"]:
            word_infos.append(word_info)
word_infos

# Display the image and overlay it with the extracted text.
plt.figure(figsize=(100, 20))
image = Image.open(BytesIO(requests.get(image_url).content))
ax = plt.imshow(image)
texts_boxes = []
texts = []
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0], bbox[1])
    patch  = Rectangle(origin, bbox[2], bbox[3], fill=False, linewidth=3, color='r')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=2, weight="bold", va="top")
#     print(bbox)
    new_box = [bbox[1], bbox[0], bbox[1]+bbox[3], bbox[0]+bbox[2]]
    texts_boxes.append(new_box)
    texts.append(text)
#     print(text)
plt.axis("off")
texts_boxes = np.array(texts_boxes)
texts_boxes

In [7]:
# Download from https://data.cityofnewyork.us/City-Government/Projected-Sea-Level-Rise/6an6-9htp directly...
data = gpd.read_file(quad_indicies_path, driver='FileGDB', layer=7)#'CellGrid_7_5Minute')

In [11]:
pd.DataFrame(data)

OSError: exception: access violation writing 0x0000000000000000

OSError: exception: access violation writing 0x0000000000000000